In [1]:
# Standard library imports
import os
from typing import List
from typing_extensions import TypedDict

# Environment variables
from dotenv import load_dotenv

# Third-party library imports
from pprint import pprint

# LangChain imports
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq

# LangGraph imports
from langgraph.graph import END, StateGraph

# Community tools
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
load_dotenv()

groq_api_key = os.environ.get("GROQ_API_KEY")
tavily_api_key = os.environ.get("TAVILY_API_KEY")

In [ ]:
### Tracing (optional)


# os.environ['LANGCHAIN TRACING V2'] = 'true'
# os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGCHAIN API KEY'] = userdata.get('LANGCHAIN_API_KEY')

In [3]:
GROQ_LLM = ChatGroq(
            model="llama-3.1-70b-versatile",
        )

In [4]:
### Search
web_search_tool = TavilySearchResults(k=1)

In [5]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in understanding financial trading preferences based on user inputs and extracting key information. Your task is to read the USER_MESSAGE and identify the following elements:
- stock_ticker: The stock symbol the user is interested in (if specified).
- risk_tolerance: The user's level of risk tolerance (e.g., Very Low, Low, Medium, High, Very High or something else).
- trading_strategy: The user's preferred trading strategy (e.g., Long-Term, Short-Term, Swing, Day Trading etc.).

If any of the above elements are not specified in the user message, return null for that element.

Provide a JSON object with a single key 'categorized_results' that contains the extracted 'stock_ticker', 'risk_tolerance', and 'trading_strategy' elements. This means your task is to return a single-key JSON object containing the 'categorized_results' key. Inside the 'categorized_results' key, there should be the keys 'stock', 'risk_tolerance', and 'trading_strategy' with the values you identified in return. Do not add any preamble or explanation beyond this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE:\n\n{user_message}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message"],
)

json_generator = prompt | GROQ_LLM | JsonOutputParser()

user_message = """I am interested in apple stock. I don't want to take a big risk but if it will benefit me in the long run, I am willing to take some risks."""

categorized_results = json_generator.invoke({"user_message": user_message})

print(categorized_results)

{'categorized_results': {'stock': 'AAPL', 'risk_tolerance': 'Medium', 'trading_strategy': 'Long-Term'}}


In [6]:
search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

As an expert in financial market data retrieval, your task is to determine the best search prompts for gathering relevant information based on the user inputs. These search prompts will be used to collect data necessary for subsequent financial analysis.

Given the USER_MESSAGE and CATEGORIZED_RESULTS which include STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, identify the search prompts that will yield the most relevant market data. Each search prompt should aim to gather specific information that will aid in financial analysis.

You should look for:
1. Current stock price and historical price trends for STOCK_TICKER.
2. Market trend predictions related to STOCK_TICKER considering the specified TRADING_STRATEGY. For example, if the specified TRADING_STRATEGY is 'Day Trading', look for the price predictions of STOCK_TICKER over 24 hours. If the specified TRADING_STRATEGY is 'Long-Term Investment', look for the price predictions of STOCK_TICKER over the long term.
3. Analysis and predictions from financial analysts for STOCK_TICKER.
4. News and recent developments affecting STOCK_TICKER.
5. Risk assessment and management strategies relevant to the specified RISK_TOLERANCE and TRADING_STRATEGY.

Provide a JSON object with a single key 'search_prompts', containing a total of 5 prompts. Do not include any preamble or explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message", "categorized_results"],
)

search_keyword_chain = search_keyword_prompt | GROQ_LLM | JsonOutputParser()

search_prompts = search_keyword_chain.invoke({"user_message": user_message, "categorized_results": categorized_results})

print(search_prompts)

{'search_prompts': ['Current stock price and historical trends of AAPL', 'Long-term price predictions for AAPL', "Financial analysts' analysis and forecasts for AAPL", 'Recent news and developments affecting AAPL', 'Risk management strategies for medium-risk long-term investments in technology stocks']}


In [7]:
# Web search
full_searches = []
for keyword in search_prompts['search_prompts']:
    print(keyword)
    temp_docs = web_search_tool.invoke({"query": keyword})
    web_results = "\n".join([d["content"] for d in temp_docs])
    web_results = Document(page_content=web_results)
    if full_searches is not None:
        full_searches.append(web_results)
    else:
        full_searches = [web_results]
print(full_searches)
print(type(full_searches))

Current stock price and historical trends of AAPL
Long-term price predictions for AAPL
Financial analysts' analysis and forecasts for AAPL
Recent news and developments affecting AAPL
Risk management strategies for medium-risk long-term investments in technology stocks
[Document(metadata={}, page_content='Historical daily share price chart and data for Apple since 1980 adjusted for splits and dividends. The latest closing stock price for Apple as of October 28, 2024 is 233.40.. The all-time high Apple stock closing price was 236.48 on October 21, 2024.; The Apple 52-week high stock price is 237.49, which is 1.8% above the current share price.; The Apple 52-week low stock price is 164.07, which\nGet a complete stock price history for Apple, starting from its first trading day. Includes open, high, low, close and volume. ... Apple Inc. (AAPL) NASDAQ: AAPL · Real-Time Price · USD. Watchlist Compare. 231.41 +0.84 (0.36%) At close: Oct 25, 2024, 4:00 PM. 230.86-.55 (-0.24%)\nFind the latest 

In [8]:
strategy_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in financial strategy analysis. Your task is to evaluate the feasibility and potential profitability of a specified trading strategy for a given stock, considering the user's risk tolerance and the provided research information.

Given the USER_MESSAGE, CATEGORIZED_RESULTS (which include STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY), and RESEARCH_INFO, assess the following:
- How suitable is the specified trading strategy (TRADING_STRATEGY) for the given stock (STOCK_TICKER)?
- What are the potential profitability and risks associated with this strategy, given the user's risk tolerance (RISK_TOLERANCE)?
- Consider historical performance, technical indicators, and trading signals from the research information (RESEARCH_INFO).

Provide a comprehensive and detailed evaluation and recommendation on whether to proceed with the trading strategy. Your analysis should include:

1. **Suitability**:
   - Assess the suitability of the TRADING_STRATEGY for STOCK_TICKER, considering the user's RISK_TOLERANCE and RESEARCH_INFO.

2. **Potential Profitability**:
   - Evaluate the potential profitability of STOCK_TICKER if the TRADING_STRATEGY is applied, considering the user's RISK_TOLERANCE and RESEARCH_INFO.

3. **Risks**:
   - Analyze the risks associated with STOCK_TICKER when applying the TRADING_STRATEGY, considering RESEARCH_INFO. Evaluate if these risks align with the user's RISK_TOLERANCE.

4. **Recommendations**:
   - Based on your analysis, provide recommendations:
     - If the user's RISK_TOLERANCE is low and the projected risks are medium or high, recommend against this investment.
     - If the user's RISK_TOLERANCE is medium and the projected risks are medium, advise the user to proceed with caution.
     - If the user's RISK_TOLERANCE is high and the projected risks are medium or low, recommend proceeding with the investment.
     - For other scenarios, use similar logic to shape your recommendation.

5. **Current and Projected Prices**:
   - Include the current price of STOCK_TICKER.
   - Provide projected price predictions based on the TRADING_STRATEGY:
     - For 'Day Trading', provide predictions for the next 24 hours.
     - For 'Long-Term Investment', provide long-term price predictions.
     - For other scenarios, use similar logic to shape your predictions.
     - If confident predictions cannot be made, explain why.

6. **Strategy Explanation**:
   - If recommending to proceed, explain the strategy in depth, including specific steps and considerations.
   - If not recommending to invest, provide a detailed explanation of the reasons and potential risks.

Be thorough and specific in your evaluation, avoiding general statements. Tailor your analysis to the specifics of the stock, strategy, and user's risk tolerance. Remember that you are an expert in analysing and developing financial strategies. You must be confident about your answers. You are the decision maker here.

Return your findings as a JSON object with a single key 'strategy_analysis' containing your evaluation and recommendation. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info"],
)

strategy_chain = strategy_prompt | GROQ_LLM | JsonOutputParser()

strategy_analysis = strategy_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches})

print(strategy_analysis)

{'strategy_analysis': {'suitability': {'analysis': "Based on the provided research information, the long-term trading strategy for Apple (AAPL) stock is suitable for the user's medium risk tolerance. The user's willingness to take some risks for long-term benefits aligns with the long-term strategy, which focuses on holding the stock for an extended period to ride out market fluctuations.", 'reasoning': "The user's medium risk tolerance and the long-term strategy's focus on stability and growth make this strategy suitable."}, 'potential_profitability': {'analysis': 'According to the research information, the average 12-month price target for AAPL is $239.44, with a high estimate of $300.00 and a low estimate of $180.00. This suggests that the stock has potential for growth in the long term. Additionally, the long-term price forecast for AAPL predicts a stock price of $299.14 by 2030, $1,221.47 by 2040, and $1,589.26 by 2050.', 'reasoning': 'The potential for growth in the short and lon

In [9]:
risk_assessment_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in financial risk assessment. Your task is to evaluate the risks associated with the specified trading strategy for the given stock, considering the user's risk tolerance, the research information, and the strategy analysis provided.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, and STRATEGY_ANALYSIS, assess the following:
- Identify and evaluate specific risks associated with the trading strategy (TRADING_STRATEGY) for the stock (STOCK_TICKER).
- Consider market volatility, potential loss, historical risk factors, and any external factors mentioned in the research information (RESEARCH_INFO).
- Assess how these risks align with the user's risk tolerance (RISK_TOLERANCE).
- Use the detailed findings from the strategy analysis (STRATEGY_ANALYSIS) to inform your risk evaluation.

Provide a comprehensive and detailed risk assessment that includes:
1. Identification of specific risks based on RESEARCH_INFO and STRATEGY_ANALYSIS.
2. The potential financial impact of these risks, using precise data from the research.
3. An assessment of how these risks align with the user's risk tolerance (RISK_TOLERANCE).
4. A final risk assessment conclusion, deciding whether to proceed with the trading strategy or not, with clear and data-backed reasoning.

Your analysis should be conclusive, clearly deciding whether the trading strategy is suitable based on the risks and the user's tolerance. Avoid general advice or vague recommendations, and provide a clear decision and explanation based on the provided data.

Return your findings as a JSON object with a single key 'risk_assessment' containing your detailed risk evaluation. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis"],
)

risk_assessment_chain = risk_assessment_prompt | GROQ_LLM | JsonOutputParser()

risk_assessment = risk_assessment_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis})

print(risk_assessment)

{'risk_assessment': {'identification_of_risks': {'specific_risks': ['Problems in China', 'Ongoing patent battle', "Potential changes to the company's business model"], 'reasoning': 'These risks are identified based on the research information provided, which highlights these factors as potential risks associated with AAPL.'}, 'potential_financial_impact': {'analysis': "The potential financial impact of these risks is difficult to quantify, but they could potentially lead to a decline in AAPL's stock price. However, the long-term strategy can help mitigate these risks by allowing the user to ride out market fluctuations and focus on the company's overall growth and performance.", 'reasoning': 'The potential financial impact of these risks is uncertain, but the long-term strategy can help mitigate them.'}, 'alignment_with_user_risk_tolerance': {'analysis': "The user's medium risk tolerance aligns with the long-term strategy, which focuses on stability and growth. The user's willingness t

In [10]:
sentiment_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in financial sentiment analysis. Your task is to evaluate the overall sentiment regarding the specified trading strategy for the given stock, considering the user's risk tolerance, the research information, the strategy analysis, and the risk assessment provided.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, STRATEGY_ANALYSIS, and RISK_ASSESSMENT, assess the following:
- Determine the overall sentiment (positive or negative) towards proceeding with the trading strategy.
- Base your evaluation on the detailed findings from the research information (RESEARCH_INFO), the strategy analysis (STRATEGY_ANALYSIS), and the risk assessment (RISK_ASSESSMENT).
- Provide a clear and concise explanation for your sentiment decision, referencing specific points from the inputs to justify your conclusion.

Your analysis should be conclusive and clearly state the overall sentiment. Avoid general advice or ambiguous recommendations. Provide a clear sentiment decision with a detailed explanation based on the provided data. You are the decision maker here.

Return your findings as a JSON object with a single key 'sentiment_analysis' containing two fields: 'sentiment' and 'explanation'. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}

RISK_ASSESSMENT: {risk_assessment}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis","risk_assessment"],
)

sentiment_chain = sentiment_prompt | GROQ_LLM | JsonOutputParser()

sentiment_analysis = sentiment_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis, "risk_assessment": risk_assessment})

print(sentiment_analysis)

{'sentiment_analysis': {'sentiment': 'Positive', 'explanation': "Based on the provided research information, strategy analysis, and risk assessment, the overall sentiment towards proceeding with the long-term trading strategy for Apple (AAPL) stock is positive. The user's medium risk tolerance aligns with the long-term strategy, which focuses on stability and growth. The potential for growth in the short and long term, as well as the increasing price targets from analysts, suggest that the strategy has potential profitability. While there are risks associated with AAPL, the long-term strategy can help mitigate these risks and focus on the company's growth and performance. Therefore, it is recommended to proceed with the long-term trading strategy for AAPL."}}


In [11]:
decision_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the Decision Agent, an expert in financial decision-making. Your task is to make a final decision on whether to proceed with the trading strategy for the given stock, considering all the previous inputs.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, STRATEGY_ANALYSIS, RISK_ASSESSMENT, and SENTIMENT_ANALYSIS, assess the following:
- Make a confident final decision on whether to proceed with the trading strategy.
- Base your decision on the detailed findings from the research information (RESEARCH_INFO), the strategy analysis (STRATEGY_ANALYSIS), the risk assessment (RISK_ASSESSMENT), and the sentiment analysis (SENTIMENT_ANALYSIS).
- Provide a concise explanation for your decision, referencing specific points from the inputs to justify your conclusion.

Your decision should be definitive and well-justified. Do not provide ambiguous recommendations. Make a clear decision with a concise and confident explanation based on the provided data. Remember, you are the decision-maker here. You should be confident about your decision.

Return your findings as a JSON object with a single key 'final_decision' containing two fields: 'decision' (whether "proceed" or "do not proceed") and 'explanation'. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}

RISK_ASSESSMENT: {risk_assessment}

SENTIMENT_ANALYSIS: {sentiment_analysis}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis","risk_assessment", "sentiment_analysis"],
)

decision_chain = decision_prompt | GROQ_LLM | JsonOutputParser()

final_decision = decision_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis, "risk_assessment": risk_assessment, "sentiment_analysis": sentiment_analysis})

print(final_decision)

{'final_decision': {'decision': 'proceed', 'explanation': "Based on the analysis, it is recommended to proceed with the long-term trading strategy for Apple (AAPL) stock. The user's medium risk tolerance aligns with the long-term strategy, which focuses on stability and growth. The potential for growth in the short and long term, as well as the increasing price targets from analysts, suggest that the strategy has potential profitability. While there are risks associated with AAPL, the long-term strategy can help mitigate these risks and focus on the company's growth and performance."}}


In [12]:
presentation_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the Presentation Agent, an expert in summarizing complex financial analyses to end users. Your task is to provide a detailed and clear explanation of the final decision on whether to proceed with the trading strategy for the given stock. This explanation should be long and thorough, starting with the final decision and explaining everything that led to this decision.

Given the USER_MESSAGE, CATEGORIZED_RESULTS which includes STOCK_TICKER, RISK_TOLERANCE, and TRADING_STRATEGY, RESEARCH_INFO, STRATEGY_ANALYSIS, RISK_ASSESSMENT, SENTIMENT_ANALYSIS, and FINAL_DECISION, create a comprehensive summary.

Your summary should:
- Start with the final decision that is stated in FINAL_DECISION.
- Explain the reasoning behind this decision in detail, referencing specific findings from the research information, strategy analysis, risk assessment, and sentiment analysis.
- Ensure that the explanation is long and thorough, providing a clear understanding of the entire decision-making process.

Return your findings as a JSON object with a single key 'summary' containing your explanation. Do not include any preamble or explanation apart from this JSON object.<|eot_id|><|start_header_id|>user<|end_header_id|>

USER_MESSAGE: {user_message}

CATEGORIZED_RESULTS: {categorized_results}

RESEARCH_INFO: {research_info}

STRATEGY_ANALYSIS: {strategy_analysis}

RISK_ASSESSMENT: {risk_assessment}

SENTIMENT_ANALYSIS: {sentiment_analysis}

FINAL_DECISION: {final_decision}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["user_message","categorized_results","research_info","strategy_analysis","risk_assessment", "sentiment_analysis","final_decision"],
)

presentation_chain = presentation_prompt | GROQ_LLM | JsonOutputParser()

summary = presentation_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": full_searches, "strategy_analysis": strategy_analysis, "risk_assessment": risk_assessment, "sentiment_analysis": sentiment_analysis, "final_decision":final_decision})

print(summary)

{'summary': "Based on the final decision, it is recommended to proceed with the long-term trading strategy for Apple (AAPL) stock. The user's medium risk tolerance aligns with the long-term strategy, which focuses on stability and growth. The potential for growth in the short and long term, as well as the increasing price targets from analysts, suggest that the strategy has potential profitability. While there are risks associated with AAPL, the long-term strategy can help mitigate these risks and focus on the company's growth and performance.\n\nThe research information provided suggests that AAPL has a strong potential for growth in the long term, with a predicted stock price of $299.14 by 2030, $1,221.47 by 2040, and $1,589.26 by 2050. The average 12-month price target for AAPL is $239.44, with a high estimate of $300.00 and a low estimate of $180.00. This suggests that the stock has potential for growth in the short term as well.\n\nThe strategy analysis also suggests that the long

In [13]:
### State
class GraphState(TypedDict):
    user_message : str
    categorized_results : str
    research_info : List[str]
    strategy_analysis : str
    risk_assessment : str
    sentiment_analysis : str
    final_decision : str
    summary : str
    num_steps : int

In [14]:
def categorize_message(state):
    """take the initial message and categorize it"""
    print("---CATEGORIZING INITIAL USER MESSAGE---")
    user_message = state['user_message']
    num_steps = int(state['num_steps'])
    num_steps += 1

    print(user_message)
    categorized_results = json_generator.invoke({"user_message": user_message})
    print(categorized_results['categorized_results'])

    return {"categorized_results": categorized_results['categorized_results'], "num_steps":num_steps}

In [15]:
def research_info_search(state):
    print("---SEARCHING THE WEB---")
    user_message = state['user_message']
    categorized_results = state['categorized_results']
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = search_keyword_chain.invoke({"user_message": user_message, "categorized_results": categorized_results})
    
    full_searches = []
    for keyword in keywords['search_prompts']:
        print(keyword)
        temp_docs = web_search_tool.invoke({"query": keyword})
        web_results = "\n".join([d["content"] for d in temp_docs])
        web_results = Document(page_content=web_results)
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]

    return {"research_info": full_searches, "num_steps":num_steps}

In [16]:
def strategy_writer(state):
    print("---STRATEGY WRITER---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    strategy_analysis = strategy_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": research_info})
    print(strategy_analysis['strategy_analysis'])

    return {"strategy_analysis": strategy_analysis['strategy_analysis'], "num_steps":num_steps}

In [17]:
def assess_risks(state):
    print("---RISK ASSESSER---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    num_steps = state['num_steps']
    num_steps += 1

    risk_assessment = risk_assessment_chain.invoke({"user_message": user_message, "categorized_results": categorized_results, "research_info": research_info, "strategy_analysis": strategy_analysis})
    print(risk_assessment['risk_assessment'])

    return {"risk_assessment": risk_assessment['risk_assessment'], "num_steps":num_steps}

In [18]:
def analyse_sentiment(state):
    print("---ANALYSE SENTIMENT ---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    risk_assessment = state["risk_assessment"]
    num_steps = state['num_steps']
    num_steps += 1

    sentiment_analysis = sentiment_chain.invoke({"user_message": user_message,
                                                 "categorized_results": categorized_results,
                                                 "research_info": research_info,
                                                 "strategy_analysis": strategy_analysis,
                                                 "risk_assessment": risk_assessment}
                                                )
    print(sentiment_analysis['sentiment_analysis'])

    return {"sentiment_analysis": sentiment_analysis['sentiment_analysis'], "num_steps":num_steps}

In [19]:
def decide_final(state):
    print("---DECIDE FINAL ---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    risk_assessment = state["risk_assessment"]
    sentiment_analysis = state["sentiment_analysis"]
    num_steps = state['num_steps']
    num_steps += 1

    final_decision = decision_chain.invoke({"user_message": user_message,
                                            "categorized_results": categorized_results,
                                            "research_info": research_info,
                                            "strategy_analysis": strategy_analysis,
                                            "risk_assessment": risk_assessment,
                                            "sentiment_analysis": sentiment_analysis}
                                           )
    print(final_decision['final_decision'])

    return {"final_decision": final_decision['final_decision'], "num_steps":num_steps}

In [20]:
def make_presentation(state):
    print("---MAKE PRESENTATION---")
    ## Get the state
    user_message = state["user_message"]
    categorized_results = state["categorized_results"]
    research_info = state["research_info"]
    strategy_analysis = state["strategy_analysis"]
    risk_assessment = state["risk_assessment"]
    sentiment_analysis = state["sentiment_analysis"]
    final_decision = state['final_decision']
    num_steps = state['num_steps']
    num_steps += 1

    summary = presentation_chain.invoke({"user_message": user_message,
                                         "categorized_results": categorized_results,
                                         "research_info": research_info,
                                         "strategy_analysis": strategy_analysis,
                                         "risk_assessment": risk_assessment,
                                         "sentiment_analysis": sentiment_analysis,
                                         "final_decision":final_decision}
                                        )
    print(summary['summary'])

    return {"summary": summary['summary'], "num_steps":num_steps}

In [21]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"User Message: {state['user_message']} \n" )
    print(f"Categorized Results: {state['categorized_results']} \n")
    print(f"Research Info: {state['research_info']} \n" )
    print(f"Strategy Analysis: {state['strategy_analysis']} \n" )
    print(f"Risk Assessment: {state['risk_assessment']} \n")
    print(f"Sentiment Analysis: {state['sentiment_analysis']} \n")
    print(f"Final Decision: {state['final_decision']} \n")
    print(f"Summary: {state['summary']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

In [22]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_message", categorize_message)
workflow.add_node("research_info_search", research_info_search)
workflow.add_node("strategy_writer", strategy_writer)
workflow.add_node("assess_risks", assess_risks)
workflow.add_node("analyse_sentiment", analyse_sentiment)
workflow.add_node("decide_final", decide_final)
workflow.add_node("make_presentation", make_presentation)

In [23]:
workflow.set_entry_point("categorize_message")
workflow.add_edge("categorize_message", "research_info_search")
workflow.add_edge("research_info_search", "strategy_writer")
workflow.add_edge("strategy_writer", "assess_risks")
workflow.add_edge("assess_risks", "analyse_sentiment")
workflow.add_edge("analyse_sentiment", "decide_final")
workflow.add_edge("decide_final", "make_presentation")
workflow.add_edge("make_presentation", END)

In [24]:
# Compile
app = workflow.compile()

In [30]:
EMAIL = """I am interested in trading apple stock. I prefer day trading, I don't want to lose my money if it is risky to enter the market at the moment."""
# EMAIL = """I am interested in trading google stock. I prefer long-term investment, I am open to take risks in short term if it is going to end in my benefit in the long run."""
# EMAIL = """I am considering investing in Tesla stock. I have a medium risk tolerance and prefer a swing trading strategy. I'm looking to capitalize on short-term price movements and hold the stock for a few weeks to a few months."""
# EMAIL = """I want to invest all my savings into penny stocks because I heard they can give high returns very quickly. I don't really have any experience with trading and I'm okay with taking big risks to try and make a lot of money fast."""
# EMAIL = """I want to invest in a stock. My risk tolerance is also medium and my trading strategy is swing trading. Could you analyze if this investment is suitable for me and provide any stock recommendations?"""

In [32]:
# Run the agent
inputs = {"user_message": EMAIL,"research_info": None, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

---CATEGORIZING INITIAL USER MESSAGE---
I am interested in trading apple stock. I prefer day trading, I don't want to lose my money if it is risky to enter the market at the moment.
{'stock': 'AAPL', 'risk_tolerance': 'Very Low', 'trading_strategy': 'Day Trading'}
'Finished running: categorize_message:'
---SEARCHING THE WEB---
Current AAPL stock price and historical price trends over the past year
AAPL day trading predictions for the next 24 hours
Financial analyst predictions for AAPL stock
Recent news and developments affecting AAPL stock
Low-risk day trading strategies for AAPL stock with stop-loss management
'Finished running: research_info_search:'
---STRATEGY WRITER---
{'suitability': "Based on the provided research information, the specified trading strategy of Day Trading for Apple stock (AAPL) may not be suitable for a user with a Very Low risk tolerance. The stock's current price is around $230, and the 12-month price forecasts by analysts have an average target of $239.44, w

In [33]:
print(output['make_presentation']['summary'])

The final decision is to not proceed with the Day Trading strategy for Apple stock (AAPL) due to the user's Very Low risk tolerance. This decision is based on a comprehensive analysis of the provided research information, strategy analysis, risk assessment, and sentiment analysis. 

The research information provided indicates that Apple stock has been performing well, with a 3-year price compound annual growth rate of 17.12% and a 12-month price forecast with an average target of $239.44. However, the stock's historical performance also suggests that it can be volatile, with a 52-week low of $164.07 and a 52-week high of $236.48.

The strategy analysis concludes that the Day Trading strategy may not be suitable for the user due to their Very Low risk tolerance. The potential risks associated with day trading, including market volatility, liquidity risks, and the potential for significant losses, may be too high for the user's comfort level. The analysis recommends considering a longer-